In [ ]:
!pip install -q paddlepaddle
!pip install -q "paddleocr>=2.0.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


ValueError: mount failed: timeout during initial read of root folder; for more info: https://research.google.com/colaboratory/faq.html#drive-timeout

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
from pprint import pprint

paddleocr = PaddleOCR(
    lang="en",
    ocr_version="PP-OCRv4",
    det_model_dir='path_to_lightweight_detection_model',
    rec_model_dir='path_to_lightweight_recognition_model',
    show_log=False,
    use_gpu=True
)

def paddle_scan(img_path_or_nparray):
    result = paddleocr.ocr(img_path_or_nparray, cls=True)
    pprint(result)
    return result


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/student_resource 3/dataset/Updated_train.csv')
df.head()

In [ ]:
import os
import glob

image_folder = "/content/drive/MyDrive/processed_images_for_OCR"
image_names = glob.glob(os.path.join(image_folder, "*.jpg"))

print(f"Found {len(image_names)} images.")


In [ ]:
len(image_names)

In [ ]:
from PIL import Image

image_path = image_names[0]
img = Image.open(image_path)


In [ ]:
img

In [ ]:
result = paddle_scan(os.path.join("/content/drive/MyDrive/projects/amazon/processed_images_for_OCR", image_names[0]))

In [ ]:
def extract_texts_from_result(result):
    extracted_texts = []


    for res in result:
        for line in res:
            detected_text = line[1][0]
            extracted_texts.append(detected_text)

    return extracted_texts


result = paddle_scan(os.path.join("/content/drive/MyDrive/projects/amazon/processed_images_for_OCR", image_names[0]))

all_texts = extract_texts_from_result(result)
print("All detected texts:", all_texts)


In [ ]:
df['extracted_text'] = df.map(lambda x: paddle_scan(os.path.join("/content/drive/MyDrive/projects/amazon")))

In [ ]:
import os
import cv2
import pandas as pd
import torch
from paddleocr import PaddleOCR
from difflib import SequenceMatcher
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from torch import nn
from sklearn.metrics import f1_score


In [ ]:

unit_variant_map = {
    'inch': ['in', "'", '”'],
    'foot': ['ft', 'feet', '′'],
    'centimetre': ['cm', 'centimeter'],
    'metre': ['m', 'meter'],
    'millimetre': ['mm', 'millimeter'],
    'yard': ['yd'],
    'gram': ['g'],
    'kilogram': ['kg'],
    'microgram': ['μg', 'mcg'],
    'milligram': ['mg'],
    'ounce': ['oz'],
    'pound': ['lb', 'lbs'],
    'ton': ['t'],
    'kilovolt': ['kV'],
    'millivolt': ['mV'],
    'volt': ['V'],
    'kilowatt': ['kW'],
    'watt': ['W'],
    'litre': ['l', 'liter'],
    'millilitre': ['ml', 'milliliter'],
    'fluid ounce': ['fl oz'],
    'gallon': ['gal'],
    'pint': ['pt'],
    'quart': ['qt']
}


entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce',
                    'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

def normalize_unit(text, unit_variant_map):
    for standard_unit, variants in unit_variant_map.items():
        for variant in variants:
            if variant in text:
                return standard_unit
    return text


In [ ]:

def extract_texts_from_result(result):
    extracted_texts = []
    for res in result:
        for line in res:
            detected_text = line[1][0]
            extracted_texts.append(detected_text)
    return extracted_texts

def find_and_normalize_units(text, entity_name, entity_unit_map, unit_variant_map):
    normalized_units = []
    normalized_text = normalize_unit(text, unit_variant_map)
    possible_units = entity_unit_map.get(entity_name, set())

    for unit in possible_units:
        if unit in normalized_text:
            normalized_units.append(unit)

    return normalized_units


In [ ]:
# for idx, row in data.iterrows():
#     img = cv2.imread(os.path.join(image_folder, f"{row['image_name']}.jpg"))
#     print(img)
#     break

None


In [ ]:
import os
import cv2
import pandas as pd

image_folder = '/content/drive/MyDrive/processed_images_for_OCR'
data = pd.read_csv('/content/drive/MyDrive/student_resource 3/dataset/Updated_train.csv')

def image_exists(image_name, image_folder):
    return os.path.exists(os.path.join(image_folder, f'{image_name}.jpg'))
data = data[data['image_name'].apply(lambda x: image_exists(x, image_folder))]

matched_data = []

for idx, row in data.iterrows():
    image_name = row['image_name']
    entity_name = row['entity_name']
    entity_value = row['entity_value']


    img_path = os.path.join(image_folder, f'{image_name}.jpg')
    img = cv2.imread(img_path)

    if img is not None:

        ocr_result = paddleocr.ocr(img)

        all_texts = extract_texts_from_result(ocr_result)

        matched_units = []
        for text in all_texts:
            units = find_and_normalize_units(text, entity_name, entity_unit_map, unit_variant_map)
            if units:
                matched_units.append(units)

        matched_data.append({
            'image_name': image_name,
            'entity_name': entity_name,
            'entity_value': entity_value,
            'matched_units': matched_units
        })

matched_data_df = pd.DataFrame(matched_data)


In [ ]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
matched_data

[]

In [ ]:

texts = [" ".join(row['matched_units']) for row in matched_data]
labels = [row['entity_value'] for row in matched_data]
X = torch.stack([get_bert_embeddings(text) for text in texts])
y = torch.tensor(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


RuntimeError: stack expects a non-empty TensorList